<h3 align="center">Codebasics ML Course: Grid Search CV</h3>

We will generate a synthetic dataset

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(
    n_features=10, 
    n_samples=1000, 
    n_informative=8,
    n_redundant=2,
    n_repeated=0,
    n_classes=2, 
    random_state=42
)

### Method 1: Evaluate the model using train, test split and tune parameters by trial and error

In [7]:
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

model = DecisionTreeClassifier(criterion="entropy", max_depth=10) # criteria: "gini" or "entropy", max_depth=5 or 10
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.83      0.78      0.80       130
           1       0.78      0.83      0.80       120

    accuracy                           0.80       250
   macro avg       0.80      0.81      0.80       250
weighted avg       0.81      0.80      0.80       250



### Method 2: Use K Fold Cross Validation

In [8]:
from sklearn.model_selection import cross_val_score

cross_val_score(DecisionTreeClassifier(criterion="gini", max_depth=5), X, y, cv=5)

array([0.78 , 0.785, 0.75 , 0.795, 0.775])

In [9]:
cross_val_score(DecisionTreeClassifier(criterion="entropy", max_depth=5), X, y, cv=5)

array([0.765, 0.785, 0.76 , 0.81 , 0.785])

In [10]:
criterion = ["gini", "entropy"]
max_depth = [5, 10, 15]

avg_scores = {}

for c in criterion:
    for d in max_depth:
        clf = DecisionTreeClassifier(criterion=c, max_depth=d)
        score_list = cross_val_score(clf, X, y, cv=5)
        avg_scores[c + "_" + str(d)] = np.average(score_list)
        
avg_scores

{'gini_5': 0.778,
 'gini_10': 0.783,
 'gini_15': 0.787,
 'entropy_5': 0.7789999999999999,
 'entropy_10': 0.784,
 'entropy_15': 0.8089999999999999}

### Method 3: Use GridSearchCV

In [11]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(
    DecisionTreeClassifier(),
    {'criterion': ["gini", "entropy"],'max_depth': [5, 10, 15]},
    cv=5,
    return_train_score=False
)
clf.fit(X, y)
clf.cv_results_

{'mean_fit_time': array([0.00357151, 0.00450606, 0.0045732 , 0.00540333, 0.00692348,
        0.00758023]),
 'std_fit_time': array([0.00045685, 0.00049501, 0.00041782, 0.00031715, 0.00051849,
        0.00063177]),
 'mean_score_time': array([0.00030355, 0.00012255, 0.00042081, 0.00019965, 0.00020409,
        0.00061212]),
 'std_score_time': array([0.00039946, 0.00024509, 0.00051564, 0.0003993 , 0.00040817,
        0.00050071]),
 'param_criterion': masked_array(data=['gini', 'gini', 'gini', 'entropy', 'entropy',
                    'entropy'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[5, 10, 15, 5, 10, 15],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'criterion': 'gini', 'max_depth': 5},
  {'criterion': 'gini', 'max_depth': 10},
  {'criterion': 'gini', 'max_depth': 15},
  {'criterion': 'entropy',

In [12]:
df = pd.DataFrame(clf.cv_results_)
df.head(3)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003572,0.000457,0.000304,0.000399,gini,5,"{'criterion': 'gini', 'max_depth': 5}",0.775,0.785,0.75,0.795,0.77,0.775,0.015166,6
1,0.004506,0.000495,0.000123,0.000245,gini,10,"{'criterion': 'gini', 'max_depth': 10}",0.810,0.740,0.80,0.775,0.81,0.787,0.026758,4
2,0.004573,0.000418,0.000421,0.000516,gini,15,"{'criterion': 'gini', 'max_depth': 15}",0.815,0.715,0.81,0.815,0.81,0.793,0.039064,2


In [13]:
df[["param_criterion", "param_max_depth", "mean_test_score"]]

,param_criterion,param_max_depth,mean_test_score
0,gini,5,0.775
1,gini,10,0.787
2,gini,15,0.793
3,entropy,5,0.782
4,entropy,10,0.791
5,entropy,15,0.811


In [14]:
clf.best_params_

{'criterion': 'entropy', 'max_depth': 15}

In [15]:
model = clf.best_estimator_
model

DecisionTreeClassifier(criterion='entropy', max_depth=15)

### Now let's try different models with different parameters

In [16]:
from sklearn import svm

model_params = {
    'decision_tree': {
        'model': DecisionTreeClassifier(),
        'params' : {
            'criterion': ["gini", "entropy"],
            'max_depth': [5, 10, 15]
        }  
    },
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']            
        }
    }
}

scores = []

for key, val in model_params.items():
    clf = GridSearchCV(
        val['model'],
        val['params'],
        cv=5,
        return_train_score=False
    )
    clf.fit(X, y)
    scores.append({
        'model': key,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
scores

[{'model': 'decision_tree',
  'best_score': 0.812,
  'best_params': {'criterion': 'entropy', 'max_depth': 15}},
 {'model': 'svm',
  'best_score': 0.9260000000000002,
  'best_params': {'C': 1, 'kernel': 'rbf'}}]

In [17]:
df = pd.DataFrame(scores, columns=["model", "best_score", "best_params"])
df

,model,best_score,best_params
0,decision_tree,0.812,"{'criterion': 'entropy', 'max_depth': 15}"
1,svm,0.926,"{'C': 1, 'kernel': 'rbf'}"
